
<img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px">

<hr style="border-width:2px;border-color:#75DFC1">
<h1 style = "text-align:center" > Introduction à TensorFlow </h1>
<h2 style = "text-align:center" > Transfer Learning </h2>
<hr style="border-width:2px;border-color:#75DFC1">

> Bienvenue dans le dernier exercice du module d'introduction à Tensorflow. Cet exercice a pour objectif de vous rappeler le fonctionnement du **transfer learning** et traiter toutes les étapes d'un problème de classification d'image.
>
> Le jeu de données est constitué d'image de radiographie contenant quatre classes : **glioma tumor**, **meningioma tumor**, **no tumor**, **pituitary tumor**. Vous trouverez plus d'informations sur le jeu de données [ici](https://www.kaggle.com/sartajbhuvaji/brain-tumor-classification-mri).
>
><img src='https://assets-datascientest.s3-eu-west-1.amazonaws.com/notebooks/introduction_tensorflow_tumors.png'>
>
>Les images se trouvent dans le dossier `Training` et `Testing`. Et, une fois dans le dossier, les images de chaque classe se trouvent dans des dossiers séparés.
>
><img src='https://assets-datascientest.s3-eu-west-1.amazonaws.com/notebooks/introduction_tensorflow_folder_image.png'>

* Exécuter la cellule suivant permettant de créer un dataframe **`df`** contenant le chemin vers l'image et la classe correspondante.

In [1]:
import glob
import pandas as pd
# Trouver tous les chemins vers les fichiers qui finissent par .jpg
liste = glob.glob('./Training/*/*.jpg')
# Extraire le label de chaque image
liste = list(map(lambda x : [x, x.split('/')[2]], liste))
# Créer un dataframe pandas
df = pd.DataFrame(liste, columns=['filepath', 'nameLabel'])
df['label'] = df['nameLabel'].replace(df.nameLabel.unique(), [*range(len(df.nameLabel.unique()))])
df.head()

,filepath,nameLabel,label
0,./Training/meningioma_tumor/m3 (67).jpg,meningioma_tumor,0
1,./Training/meningioma_tumor/m1(122).jpg,meningioma_tumor,0
2,./Training/meningioma_tumor/m2 (81).jpg,meningioma_tumor,0
3,./Training/meningioma_tumor/m3 (77).jpg,meningioma_tumor,0
4,./Training/meningioma_tumor/m (192).jpg,meningioma_tumor,0


## Exploration des données

> Maintenant que la correspondance entre chemin et classe est faite, intéressons-nous maintenant aux images.
>
> La fonction `read_file` de **`tensorflow.io`** permet de charger l'information brute d'un fichier (image, fichier audio, dossier texte ...). Ensuite, la fonction `decode_jpeg` de **`tensorflow.image`** permet de décoder l'information brute en un tensor de type `uint8` ou `uint16` :
>
> ```python
># Charger l'information brute en mémoire
>im = tf.io.read_file(filepath)
># Decoder l'information en un tensorflow RGB (3 channels).
>im = tf.image.decode_jpeg(im, channels=3)
>
>```
>
> <div class="alert alert-info">
<i class="fa fa-info-circle"></i> &emsp; 
On peut utiliser le package `matplotlib` ou `OpenCV` pour charger les images, mais, les fonctions de tensorflow sont plus performantes quand ils sont utilisés dans l'architecture du modèle.
</div>

* Charger une des images de notre dataframe.


* Afficher l'image à l'aide de la fonction `imshow` de **`matplotlib.pyplot`**.

> Les modèles en Deep learning n'accepte en générale que des entrées de même dimension, il est alors nécessaire de redimensionner toutes les images pour qu'elles aient la même taille. D'après la littérature, la dimension optimale entre perte d'information et complexité est aux alentours  de (256,256).
>
> La fonction `resize` de **`tensorfow`** permet de redimensionner une image en dimension `size`:
>
>```python
># Redimensionner l'image en (256,256)
>im = tf.image.resize(im, size=(256,256))
>```

* Redimensionner l'image chargée précédemment en (256,256).

## Charger le jeu de données

> Le modèle a besoin pour s'entraîner d'images/labels. Ces images peuvent soit être tous chargées initialement ou soit être chargées quand le modèle en a besoin. Il arrive que le jeu de données soit constitué de centaines de millier d'images, dans ce cas, il n'est pas concevable de les charger au préalable en mémoire (nécessite de 40Go de RAM pour 150k images 256x256x3).
>
> Dans ce cas, deux solutions sont disponibles :
>
>>* Comme le jeu de données est trop lourd, choisir un sous-échantillon (ex : 30 000 images) et le charger en mémoire.
>>
>>
>>* Charger les images pendant l'entraînement à l'aide d'un générateur. Solution à privilégier si le pré-processing n'est pas trop lourd.
>
> Les deux solutions fonctionnent, dans cet exercice nous allons **choisir la deuxième**. Comme le jeu de test est généralement plus petit, il peut être chargé en mémoire.
>
> <div class="alert alert-info">
<i class="fa fa-info-circle"></i> &emsp; 
La structure des données nous permettrait d'utiliser les instances `ImageDataGenerator` de <b>tensorflow.keras.preprocessing.image</b> pour charger les données quand le modèle en a besoin. Mais, cette instance reste très limitée, convient généralement que pour des problèmes de classification.
</div>

* Séparer le jeu de données **`df.filepath`** et la variable cible **`df.label`** en un ensemble d'entraînement **X_train_path**, **y_train**, et en un ensemble de test **X_test_path**, **y_test**. Nous choisirons un rapport de 80% pour les données d'entraînements et une graine aléatoire 1234.


* Charger les images de **X_test_path** redimensionnées à [256,256,3]  en mémoire dans la variable **X_test**.

In [1]:
from tqdm import tqdm


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

X_train_path, X_test_path, y_train, y_test = train_test_split(df.filepath, df.label, test_size = 0.2, random_state = 1234)

X_test = []
for filepath in tqdm(X_test_path):
    # Read the file
    im = tf.io.read_file(filepath)
    # Decode the file
    im = tf.image.decode_jpeg(im, channels=3)
    # Resizing
    im = tf.image.resize(im, size=(256,256))
    X_test.append([im])
    
X_test = tf.concat(X_test, axis=0)

100%|██████████| 574/574 [00:01<00:00, 362.70it/s]


> Maintenant les données de test chargées, il est nécessaire de définir un **dataset** permettant charger les images à chaque itération du modèle. Pour optimiser le temps de chargement/pré-traitement, il est possible de paralléliser chaque opération en utilisant une structure multi-tasking.
>
> Les objet de type **dataset** sur tensorflow sont capables de le faire à l'aide de l'argument **num_parallel_calls** de la méthode `map`.
>
> Pour rappel, le constructeur `from_tensor_slices` du package **tensorflow.data.Dataset** permet de convertir une liste d'array en un dataset.
>
> ```python
dataset = tf.data.Dataset.from_tensor_slices((X_path, y))
> ```
> La méthode `map` du dataset permet d'appliquer une opération à chaque observation. Exemple :
>
> ```python
dataset = dataset.map(lambda x, y : [load_image(x), y])
> ```
>
> La méthode `batch` du dataset permet de regrouper les observations sous forme de batch.
>
><img src="https://datascientest.fr/train/assets/tensorflow_02_batch.png" style="width:400px">
>
> ```python
dataset = dataset.batch(batch_size)
> ```

* Définir une fonction `load_image` avec comme argument `filepath` retournant une image redimensionnée en (256,256).


* Définir un dataset **`dataset_train`** de **`(X_train_path, y_train)`** à l'aide de la fonction `from_tensor_slices`.


* À l'aide de la méthode `map`, appliquer la fonction `load_image` à chaque valeur de **X_train_path**. Pour que le chargement s'effectue en multi-tasking, préciser l'argument `num_parallel_calls` égale à -1.


* Regrouper les observations sous forme de batch de taille 32.

> Les modèles de classification d'image ou de détection d'objet utilise généralement une approche par transfer Learning.

### Quelques rappels sur le transfer leaning :

> L'apprentissage par transfert est le phénomène par lequel un apprentissage nouveau est facilité grâce aux apprentissages antérieurs partageant des similitudes. Par exemple, les connaissances acquises lors de l’apprentissage de la reconnaissance des voitures peuvent s’appliquer lorsqu’on essaie de reconnaître des camions.
>
> Les modèles existants (VGG, ResNet, ...) sont composés de deux grandes parties. La première appelée **backbone** est un ensemble de convolution permettant l'**extraction des features de l'image**. La seconde est une succession de dense layer qui a pour but de classifier.
>
> Les données du nouveau problème doit être assez semblable avec le jeu de données utilisé pour le pré-entrainement. Dans ce cas, la méthode de transfer learning consiste à utiliser le **backbone** d'un modèle pré-entraîné comme extraction de features. Ensuite des couches `Dense` sont ajouté pour traiter le problème de classification ou de regression.

In [18]:
%matplotlib inline
from interaction_tl import show_tl
show_tl()

Using TensorFlow backend.


> Lors du début de l'apprentissage, il est nécessaire de "freezer" (bloquer) les poids de la partie pré-entrainée (**backbone**) puisqu'ils sont proches des poids optimaux. Puis, au courant de l'entraînement, on peut "unfreeze" les couches pour affiner les poids du modèle : cet opération est appelée le **fine-tuning**.
>
> <img src="https://datascientest.fr/train/assets/python_keras_picasso_unfreeze1.png" style="width:800px">
>
> Dans cet exercice, le modèle pré-entraîné sera le EfficientNet puisqu'il a montré de très bon résultat et de très bonne propriété pour le transfer learning.
>
> Voici un exemple pour charger et freeze les poids d'une modèle pré-entraîné :
>
> ```python
vgg16 = VGG16(include_top=False, input_shape=(256,256,3))
for layer in vgg16.layers:
    layer.trainable = False
model = Sequential()
model.add(vgg16)
>```

* Charger le modèle `EfficientNetB1` de **`tensorflow.keras.applications`** sous le nom **`efficientNet`**. La partie classification ne sera pas prise et l'`input_shape` sera (256,256,3).


* Freezer les poids du modèle.


* Afficher le résumé du modèle.

### Partie Classification

* Ajouter le modèle pré-entraîné à un objet `Sequential` qui portera le nom de **model**.


* Ajouter à ce modèle une couche `GlobalAveragePooling2D`.


* Puis, ajouter quelques couches `Dense` et `Dropout`.


* Finir par une couche `Dense` avec 4 neurones et une fonction activation 'softmax'.


* Afficher le résumé du modèle.

* Compiler le modèle avec votre fonction de perte `sparse_categorical_crossentropy`, un optimizer `'adam'` et une métrique `['accuracy']`.

## Entraînement du modèle


* Entraîner le modèle à l'aide la méthode `fit` sur 10 epochs en ajoutant les callbacks suivants :
    * Ajouter une sauvegarde des poids à chaque epoch à l'aide du callbacks `ModelCheckpoint`.
    
    * Diminuer le learning rate si la métrique de validation ne s'améliore pas dans les 5 dernières époques.

$\DeclareMathOperator*{\argmax}{argmax}$

## Évaluation

> La fonction d'activation **softmax** utilisée à la fin de notre modèle, fait que celui-ci retourne pour chaque image une probabilité de prédiction de chaque classe.
>
> $$ \mathbf{P(\ y\ |\ X\ )} =
\begin{bmatrix}
    P(y=0|X)  \\
    P(y=1|X)  \\
    \vdots  \\
    P(y=9|X) 
\end{bmatrix}$$
>
> Pour prédire la classe de l'image, il suffit alors de trouver pour quelle classe la probabilité est maximale :
>
> $$ \hat{y} = \argmax_{y}(P(\ y\ |\ X\ ) $$ 

* Prédire la probabilité des classes du jeu de données **X_test**.


* Prédire dans **y_pred** la classe la plus probable à l'aide de la fonction `argmax` de **`tensorflow`** en précisant `axis=-1`.

* Afficher le score de précision à l'aide de la fonction `accuracy_score` de **`sklearn.metrics`**.


* Afficher la matrice de confusion à l'aide de la fonction `confusion_matrix` de **`sklearn.metrics`**.

* Exécuter la cellule suivante pour afficher les prédictions de notre modèle sur 3 images.

<hr style="border-width:2px;border-color:#75DFC1">
<h2 style = "text-align:center" > Ce qu'il faut retenir de ce module </h2> 
<hr style="border-width:2px;border-color:#75DFC1">


## Méthodologie

> Globalement, la méthodologie pour résoudre un problème à l'aide d'outil de deep learning :
>
>1. Définir un **dataset** permettant de mettre en forme les données et de partitionner en batchs.
>
>
>2. Construire un modèle : MLP, CNN, RNN, transfer learning ...
>
>
>3. Compiler le modèle : définition d'une fonction de perte, métrique, optimizer.
>
>
>4. Entraîner le modèle, il y'a deux manières équivalentes de le faire:
>    - Méthode **`fit`**: problème simple.
>    - Calculer le gradient de la fonction de coût puis rétropropager l'erreur: problème complexe.
>    
>    
>5. Prédiction et évaluation du modèle.



## Dataset

> Tensorflow possède un sous ensemble **`tensorflow.data.dataset`**  qui permet d'appliquer toutes les étapes de pré-processing sur les données. C'est un pipeline d'opération appliqué sur les entrées. Un exemple :
>
> * Charger les images + redimensionner (en paralélisant les calculs).
>
>
> * Appliquer des méthodes d'augmentation de données.
>
>
> * Normaliser nos données.
>
>
> * Les regrouper en lot de données.



## Keras

> La version de tensorflow 2.0+ a été construite autour du framework [**keras**](https://www.tensorflow.org/guide/keras).
>
> Vous pouvez retrouver toutes les fonctionnalités de **keras** dans **`tensorflow.keras`**. Il y'a notamment :
>
>* Les couches de neurones dans **`tensorflow.keras.layers`**. Il est possible de créer des couches personnalisées en faisant hériter la classe de la couche avec **`tensorflow.keras.layers.Layer`**.
>
>
>* Les modèles pré-entrainés dans  **`tensorflow.keras.applications`**.
>
>
>* Les fonctions de pertes dans **`tensorflow.keras.losses`**
>
>
>* Les métriques dans **`tensorflow.keras.metrics`**.
>
>
>* Les optimizers dans **`tensorflow.keras.optimizers`**.

## Callbacks


> Les rappels (***callbacks***) sont des outils qui permettent de contrôler l'entraînement et évaluation d'un modèle. Il est alors possible de connaître l'état interne d'un modèle, de le sauvegarder, d'afficher des statistiques intéressantes et même de changer des hyperparamètres pendant les étapes de l'entraînement.
>
> Les callbacks suivants peuvent être très pratique en Deep Learning :
>
> * Sauvegarder les meilleurs poids du modèle au cours de l'entraînement :
>
>```python
>callbacks.ModelCheckpoint(filepath = filepath, 
>                           monitor = 'val_loss',
>                           save_best_only = True,
>                           save_weights_only = False,
>                           mode = 'min',
>                           save_freq = 'epoch')
>
>```
>
> * Réduire automatiquement le learning rate :
>
>```python
>callbacks.ReduceLROnPlateau(monitor = 'val_loss',
>                             patience=5,
>                             factor=0.5,
>                             verbose=2,
>                             mode='min')
>```
>
> * Arrêter l'entraînement si le modèle n'évolut plus. Très pratique pour ne pas gérer le nombre d'époque et laisser le modèle s'arrêter quand il n'évolut plus.
>
>```python
>callbacks.EarlyStopping(monitor = 'val_loss',
>                         patience = 8,
>                         mode = 'min',
>                         restore_best_weights = True)
>```